In [ ]:
%run femshape
import matplotlib.pyplot as plt
import numpy as np
from numpy import cos, sin, pi
from numpy.linalg import norm

## Create some planar curves

In [ ]:
n = 20000
t = np.linspace(0,2*pi,n,endpoint=False)
gamma1_highres = np.vstack((0.9*cos(t),0.4*sin(t))).T

n = 1000
t = np.linspace(0,2*pi,n,endpoint=False)
gamma1_lowres = np.vstack((0.9*cos(t),0.4*sin(t))).T

n = 5000
t = np.linspace(0,2*np.pi,n,endpoint=False)
gamma1 = np.vstack((0.9*cos(t),0.4*sin(t))).T

gamma1_repar1 = np.roll(gamma1,n//4,axis=0)
gamma1_repar2 = np.vstack((0.9*cos(t+pi/2),0.4*sin(t+pi/2))).T
gamma1_repar3 = np.vstack((0.9*cos(t+0.01*sin(3*t)),0.4*sin(t+0.01*sin(3*t)))).T

reg_pert = np.vstack((cos(6*t),sin(7*t))).T
rand_pert = np.random.randn(n,2)

gamma2 = np.vstack((0.7*cos(t),0.7*sin(t))).T

plt.plot(gamma1[:,0],gamma1[:,1],gamma2[:,0],gamma2[:,1])
plt.axis('equal')

## Create the invariants calculation object

In [ ]:
space = Space(order=2, meshsize=64)

## Compute the invariants

In [ ]:
curves = [gamma1, gamma2, gamma1_repar1, gamma1_repar2, gamma1_repar3, gamma1_highres, gamma1_lowres]
curve_invariants = [CurveInvariant(space, curve) for curve in curves]

## Compute relative errors of perturbations

### Check approximate invariance under reparameterization

In [ ]:
inv1, inv2, inv1_repar1, inv1_repar2, inv1_repar3, inv1_highres, inv1_lowres = [ci.invariants for ci in curve_invariants]

In [ ]:
print("Relative norm between invariants for gamma1 and gamma2 is %.4e"%(norm(inv1-inv2)/norm(inv1)))
print("Relative norm between invariants for gamma1 and gamma1_repar1 is %.4e"%(norm(inv1-inv1_repar1)/norm(inv1)))
print("Relative norm between invariants for gamma1 and gamma1_repar2 is %.4e"%(norm(inv1-inv1_repar2)/norm(inv1)))
print("Relative norm between invariants for gamma1 and gamma1_repar3 is %.4e"%(norm(inv1-inv1_repar3)/norm(inv1)))
print("Relative norm between invariants for gamma1 and gamma1_highres is %.4e"%(norm(inv1-inv1_highres)/norm(inv1)))
print("Relative norm between invariants for gamma1 and gamma1_lowres is %.4e"%(norm(inv1-inv1_lowres)/norm(inv1)))

### Check sensitivity w.r.t. perturbations

In [ ]:
errvec_reg = []
errvec_rand = []
epsvec = np.logspace(-8,-1,10)
for epsilon in epsvec:
    errvec_reg.append(norm(inv1-CurveInvariant(space, gamma1+epsilon*reg_pert).invariants)/norm(inv1))
    errvec_rand.append(norm(inv1-CurveInvariant(space, gamma1+epsilon*rand_pert).invariants)/norm(inv1))
    #print(errvec[-1])

In [ ]:
plt.loglog(epsvec,errvec_reg,epsvec,errvec_rand,epsvec,epsvec)
plt.xlabel('epsilon')
plt.ylabel('relative error')
plt.legend(('regular pert','random pert','linear growth'),loc='upper left')

### Relative errors for other function spaces

In [ ]:
shapecalc_loword = Space(order=1, meshsize=64)
shapecalc_highord = Space(order=8, meshsize=64)
shapecalc_fine = Space(order=2, meshsize=256)
shapecalc_rough = Space(order=8, meshsize=8)

In [ ]:
calc = shapecalc_rough
inv11 = CurveInvariant(calc, gamma1).invariants
errvec2_reg = []
errvec2_rand = []
epsvec2 = np.logspace(-8,-1,10)
for epsilon in epsvec2:
    errvec2_reg.append(norm(inv11-CurveInvariant(calc, gamma1+epsilon*reg_pert).invariants)/norm(inv11))
#     print(errvec2_reg[-1])
plt.loglog(epsvec2,errvec2_reg,epsvec2,epsvec2)
plt.xlabel('epsilon')
plt.ylabel('relative error')
plt.legend(('regular pert','linear growth'),loc='upper left')

In [ ]:
calc.V.dim()

## Visualize the invariants through FEniCS functions

In [ ]:
calc = shapecalc_loword
ci = CurveInvariant(calc, gamma1)
(xmat,ymat) = ci.matrix_representation(size=256)
plt.imshow(xmat)

In [ ]:
plt.imshow(ymat)